# 펑션 콜링 적용하기
타임존을 기반으로 현재 시간을 반환하는 실습 진행
- 타임존이란 ? 전 세계적으로 시간대가 다르기 때문에, 특정 지역의 시간을 정확히 계산하기 위해서는 타임존 설정 필요

## 1. 필요 라이브러리 불러오기
`datetime`과 `pytz` 라이브러리를 사용하여 타임존을 설정하고 현재 시간을 가져올 수 있음

In [ ]:
from datetime import datetime
import pytz

## 2. GPT에서 사용할 함수 정의하기
타임존 기반 현재 시간을 반환하는 함수 정의

### get_current_time 함수 설명
- `timezone` 매개변수: 타임존을 문자열로 입력받음. 기본값은 `Asia/Seoul`
- `pytz.timezone(timezone)`: 입력받은 타임존에 맞는 시간대 설정
- `datetime.now(tz)`: 설정된 타임존의 현재 시간을 가져옴
- `strftime`: 시간을 `YYYY-MM-DD HH:MM:SS` 형식의 문자열로 변환

In [ ]:
def get_current_time(timezone: str = 'Asia/Seoul'):
    tz = pytz.timezone(timezone) # 타임존 설정
    now = datetime.now(tz).strftime("%Y-%m-%d %H:%M:%S") # 설정한 타임존의 현재 시간을 다음 시간 포맷과 같이 반환 ex) 2025-10-27 20:00:00
    now_timezone = f'{now} {timezone}'
    print(now_timezone)
    return now_timezone

## 3. 함수 실행해보기
아래 코드를 실행하여 `America/New_York` 타임존의 현재 시간 확인

In [3]:
get_current_time('America/New_York')

2025-10-27 22:03:58 America/New_York


'2025-10-27 22:03:58 America/New_York'

## 4. GPT를 위해 사용할 함수 설명 추가하기
GPT API 호출 시 펑션 콜링할 도구를 정의해야 함

콜링할 함수의 메타데이터를 JSON 형식으로 정의하며, 도구의 이름, 설명, 매개변수 등을 포함하고 있음

### 도구 추가 설명
- `type`: 이 도구가 함수임을 나타냄
- `name`: 함수 이름 정의
- `description`: 함수의 역할 설명
- `parameters`: 함수가 받을 매개변수의 형식과 설명 정의

In [ ]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_time",
            "description": "해당 타임존의 날짜와 시간을 반환합니다.",
            "parameters": {
                "type": "object",
                "properties": {
                    'timezone': {
                        'type': 'string',
                        'description': '현재 날짜와 시간을 반환할 타임존을 입력하세요. (예: Asia/Seoul)',
                    },
                },
                "required": ['timezone'],
            },
        }
    },
]

## 5. 코드 실행
아래 코드를 실행하여 전체 코드 테스트

In [5]:
if __name__ == '__main__':
    get_current_time('America/New_York')

2025-10-27 22:03:58 America/New_York


# 6. GPT 실행
- GPT 실행을 위해 필요한 라이브러리 불러오기

In [6]:
from openai import OpenAI
from dotenv import load_dotenv
import os
import json

In [ ]:
api_key = =  # 환경 변수에서 API 키 가져오기

client = OpenAI(api_key=api_key)  # 오픈AI 클라이언트의 인스턴스 생성

## GPT에 tools 정보 포함하기
- 기존 `get_ai_response` 함수 파라미터에 tools 추가
- `tools`에 gpt_functions의 `tools` 대입해주면 위에서 정의한 `get_current_time` 함수를 사용할 수 있음

In [9]:
def get_ai_response(messages, tools=None):
    response = client.chat.completions.create(
        model="gpt-4o",  # 응답 생성에 사용할 모델 지정
        messages=messages,  # 대화 기록을 입력으로 전달
        tools=tools,  # 사용 가능한 도구 목록 전달
    )
    return response  # 생성된 응답 내용 반환

## get_ai_response 함수 실행 준비하기
아래 코드는 사용자 입력을 받아 GPT와 대화를 진행하는 내용

- `input`: 사용자로부터 입력을 받음
- `exit`: 사용자가 "exit"을 입력하면 대화를 종료
- `messages`: 대화 맥락을 유지할 수 있도록 계속 append 시켜줌, 사용자와 GPT의 메시지를 저장함

### `tool_calls` 처리

GPT가 도구를 호출해야 할 때 `tool_calls`가 포함됨

- `tool_calls`?
  - GPT가 특정 작업을 수행하기 위해 도구를 호출해야 한다고 판단하면 응답에 `tool_calls`가 포함됨

  - ex) 사용자가 "현재 시간을 알려줘"라고 요청하면 GPT는 `get_current_time` 함수를 호출해야 한다고 판단할 수 있음

- `tool_calls` 처리 과정
  - `tool_calls`가 있는지 확인

  - 호출해야 할 함수 이름과 매개변수 추출

  - 해당 함수를 실행하고 결과를 대화 기록에 추가

- 다음 셀에서 실행한 질문들
  - 1) 안녕하세요
  - 2) 신기한 문장을 생성해보세요
  - 3) 현재 서울의 시간은 몇시인가요?
  - 4) 현재 태국 방콕의 시간은 몇시인가요?
  - 5) 아름다운 문장을 생성해보세요
  - 6) exit

In [10]:
messages = [
    {"role": "system", "content": "너는 사용자를 도와주는 상담사야."},  # 초기 시스템 메시지
]

while True:
    user_input = input("사용자\t: ")  # 사용자 입력 받기

    if user_input == "exit":  # 사용자가 대화를 종료하려는지 확인
        break

    messages.append({"role": "user", "content": user_input})  # 사용자 메시지 대화 기록에 추가

    ai_response = get_ai_response(messages, tools=tools)
    ai_message = ai_response.choices[0].message
    print(ai_message)  # gpt에서 반환되는 값을 파악하기 위해 임시로 추가

    tool_calls = ai_message.tool_calls  # AI 응답에 포함된 tool_calls를 가져옴

    if tool_calls:  # tool_calls가 있는 경우
        for tool_call in tool_calls:
            tool_name = tool_call.function.name # 실행해야한다고 판단한 함수명 받기
            tool_call_id = tool_call.id         # tool_call 아이디 받기
            arguments = json.loads(tool_call.function.arguments) # 문자열을 딕셔너리로 변환

            if tool_name == "get_current_time":  # 만약 tool_name이 "get_current_time"이라면
                messages.append({
                    "role": "function",  # role을 "function"으로 설정
                    "tool_call_id": tool_call_id,
                    "name": tool_name,
                    "content": get_current_time(timezone=arguments['timezone']),  # 타임존 추가
                })

        messages.append({"role": "system", "content": "이제 주어진 결과를 바탕으로 답변할 차례다."})  # 함수 실행 완료 메시지 추가
        ai_response = get_ai_response(messages, tools=tools) # 다시 GPT 응답 받기
        ai_message = ai_response.choices[0].message

    messages.append(ai_message)  # AI 응답을 대화 기록에 추가하기

    print("AI\t: " + ai_message.content)  # AI 응답 출력

ChatCompletionMessage(content='안녕하세요! 무엇을 도와드릴까요?', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None, annotations=[])
AI	: 안녕하세요! 무엇을 도와드릴까요?
ChatCompletionMessage(content='창의적인 아이디어를 생성하기 위한 몇 가지 방법을 소개해드릴게요. 다음 방법들을 활용해보세요:\n\n1. **랜덤 자극 사용**: 사전이나 인터넷에서 무작위로 단어를 선택한 다음, 그 단어를 활용해 새로운 해결책이나 아이디어를 떠올려 보세요.\n\n2. **SCAMPER 기법**: 기존 아이디어나 제품을 기반으로 다음 질문을 던져보세요:\n   - 대체할 수 있는 것은 무엇인가? (Substitute)\n   - 결합할 수 있는 것은 무엇인가? (Combine)\n   - 적응할 수 있는 것은 무엇인가? (Adapt)\n   - 수정, 확장, 축소할 수 있는 것은 무엇인가? (Modify)\n   - 다른 용도로 사용할 수 있는 것은 무엇인가? (Put to another use)\n   - 제거할 수 있는 것은 무엇인가? (Eliminate)\n   - 재배열할 수 있는 것은 무엇인가? (Rearrange)\n\n3. **마인드맵 작성**: 중심 주제를 설정하고 그것과 관련된 아이디어를 가지를 뻗듯이 계속해서 확장해보세요.\n\n4. **브레인스토밍**: 제한 없이 모든 아이디어를 나열하고, 평가하지 않고 양을 늘려보는 시간을 가져보세요.\n\n5. **아날로그 탐색**: 다른 분야나 산업에서 사용되는 방법이나 도구를 참고하여 유사한 문제를 해결해보세요. 예를 들어, 자연에서 영감을 받아 새로운 디자인을 만들어보는 것입니다.\n\n6. **이야기 만들기**: 문제를 사람이 주인공인 이야기로 꾸며서 해결책을 찾아보세요.\n\n7. **시각적 사고 도구 활용**: 그림이나 차트를 사용하여 문제를 시각적으